In [1]:

! pip install python-dotenv
! pip install -qU langchain-openai
! pip install -qU langchain_community beautifulsoup4
! pip install faiss-cpu

In [2]:
env_content = """
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT=https://api.smith.langchain.com
LANGSMITH_API_KEY=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
LANGSMITH_PROJECT=pr-minty-canvas-57
OPENAI_API_KEY=
"""

with open(".env", "w") as f:
    f.write(env_content.strip())

print(".env file created successfully!")

.env file created successfully!


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/how-captide-is-redefining-equity-research-with-agentic-workflows-built-on-langgraph-and-langsmith/")

In [5]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://blog.langchain.dev/how-captide-is-redefining-equity-research-with-agentic-workflows-built-on-langgraph-and-langsmith/', 'title': 'How Captide is redefining equity research with agentic workflows running on LangGraph Platform', 'language': 'en'}, page_content="\n\n\nHow Captide is redefining equity research with agentic workflows running on LangGraph Platform\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHow Captide is redefining equity research with agentic workflows running on LangGraph Platform\n\nCase Studies\n3 min read\nJan 20, 2025\n\n\n\n\n\nCaptideâ€™s platform transforms how investment research teams work with financial data. By automating the extraction of insights and metrics fr

In [6]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [7]:
documents

[Document(metadata={'source': 'https://blog.langchain.dev/how-captide-is-redefining-equity-research-with-agentic-workflows-built-on-langgraph-and-langsmith/', 'title': 'How Captide is redefining equity research with agentic workflows running on LangGraph Platform', 'language': 'en'}, page_content='How Captide is redefining equity research with agentic workflows running on LangGraph Platform\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHow Captide is redefining equity research with agentic workflows running on LangGraph Platform\n\nCase Studies\n3 min read\nJan 20, 2025'),
 Document(metadata={'source': 'https://blog.langchain.dev/how-captide-is-redefining-equity-research-with-agentic-workflows-built-on-langgraph-and-langsmith/'

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")

In [9]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [10]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"and gather invaluable user feedback as soon as tasks are run.Captideâ€™s integration of LangSmith allows for precise monitoring of agent workflows, with detailed traces that highlight response times, error rates, and operational costs. This visibility ensures that the platform maintains its high standards of performance and reliability.LangSmith has also been crucial when incorporating thumbs-up and thumbs-down options within the platform where users can directly rate the quality of outputs. This feedback is collected and analyzed, creating a growing dataset that helps refine agent behavior and improve system performance over time. With LangSmithâ€™s evaluation tools, this feedback loop enables Captide to identify trends, address weaknesses, and continuously enhance the user experience.Deploying on LangGraph PlatformWhen LangChain launched LangGraph Platform this summer, it was a no-brainer to deploy their cutting-edge agents on it.With Captide's agent built on LangGraph, it was a"

In [11]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [12]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)
llm = ChatOpenAI(model="gpt-4o")

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cff32246790>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cff3314e550>, root_client=<openai.OpenAI object at 0x7cff332c0b10>, root_async_client=<openai.AsyncOpenAI object at 0x7cff32246a90>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'},

In [13]:
from langchain_core.documents import Document

doc = Document(page_content="By allowing users to articulate complex analysis tasks in natural language, Captide simplifies the process of extracting financial metrics")

response = document_chain.invoke({
    "input": "Redefining Financial Analysis with NLP Workflows",
    "context": [doc]
})

print(response)


Captide simplifies the process of extracting financial metrics by allowing users to articulate complex analysis tasks in natural language.


In [14]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [15]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Redefining Financial Analysis with NLP Workflows"})
response['answer']

"Captide is redefining equity research through the use of its platform, which leverages agentic workflows running on the LangGraph Platform. By automating the extraction of insights and metrics from financial documents such as regulatory filings and investor relations documents, Captide allows analysts to create customized datasets and conduct analyses more efficiently. The use of NLP (Natural Language Processing) workflows enables users to specify complex analysis tasks in natural language, which Captide's agents then execute, automating the entire data retrieval and processing pipeline. This process overcomes the fixed-schema limitations of legacy platforms, allowing for flexibility and the analysis of larger volumes of data. Additionally, LangGraph provides parallel processing capabilities and structured outputs, further enhancing operational efficiency and precision in analysis."

In [16]:
response

{'input': 'Redefining Financial Analysis with NLP Workflows',
 'context': [Document(id='a3ba51fa-e485-4b83-bdfc-c8ff8b338e20', metadata={'source': 'https://blog.langchain.dev/how-captide-is-redefining-equity-research-with-agentic-workflows-built-on-langgraph-and-langsmith/', 'title': 'How Captide is redefining equity research with agentic workflows running on LangGraph Platform', 'language': 'en'}, page_content='Captideâ€™s platform transforms how investment research teams work with financial data. By automating the extraction of insights and metrics from regulatory filings and investor relations documents, analysts can create customized datasets and analyses with extreme efficiency. At the heart of this innovation is their commitment to NLP workflows and its strategic integration of LangGraph and LangSmith, hosted on LangGraph Platform.Redefining Financial Analysis with NLP WorkflowsBy allowing users to articulate complex analysis tasks in natural language, Captide simplifies the proc